In [10]:
import pandas as pd
import numpy as np
products = pd.read_csv('/kaggle/input/amazon-products-sales-dataset-42k-items-2025/amazon_products_sales_data_cleaned.csv')



In [12]:
products.isna().sum()

product_title               0
product_rating           1024
total_reviews            1024
purchased_last_month    10511
discounted_price         2062
original_price           2062
is_best_seller              0
is_sponsored                0
has_coupon                  0
buy_box_availability    14653
delivery_date           11983
sustainability_tags     39267
product_image_url           0
product_page_url         2069
data_collected_at           0
product_category            0
discount_percentage      2062
dtype: int64

In [14]:
products.dtypes

product_title            object
product_rating          float64
total_reviews           float64
purchased_last_month    float64
discounted_price        float64
original_price          float64
is_best_seller           object
is_sponsored             object
has_coupon               object
buy_box_availability     object
delivery_date            object
sustainability_tags      object
product_image_url        object
product_page_url         object
data_collected_at        object
product_category         object
discount_percentage     float64
dtype: object

In [15]:
products.head()

,product_title,product_rating,total_reviews,purchased_last_month,discounted_price,original_price,is_best_seller,is_sponsored,has_coupon,buy_box_availability,delivery_date,sustainability_tags,product_image_url,product_page_url,data_collected_at,product_category,discount_percentage
0,BOYA BOYALINK 2 Wireless Lavalier Microphone f...,4.6,375.0,300.0,89.68,159.00,No Badge,Sponsored,Save 15% with coupon,Add to cart,2025-09-01,Carbon impact,https://m.media-amazon.com/images/I/71pAqiVEs3...,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,2025-08-21 11:14:29,Phones,43.60
1,"LISEN USB C to Lightning Cable, 240W 4 in 1 Ch...",4.3,2457.0,6000.0,9.99,15.99,No Badge,Sponsored,No Coupon,Add to cart,2025-08-29,NaN,https://m.media-amazon.com/images/I/61nbF6aVIP...,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,2025-08-21 11:14:29,Laptops,37.52
2,"DJI Mic 2 (2 TX + 1 RX + Charging Case), Wirel...",4.6,3044.0,2000.0,314.00,349.00,No Badge,Sponsored,No Coupon,Add to cart,2025-09-01,NaN,https://m.media-amazon.com/images/I/61h78MEXoj...,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,2025-08-21 11:14:29,Laptops,10.03
3,"Apple AirPods Pro 2 Wireless Earbuds, Active N...",4.6,35882.0,10000.0,162.24,162.24,Best Seller,Organic,No Coupon,NaN,NaN,NaN,https://m.media-amazon.com/images/I/61SUj2aKoE...,https://www.amazon.com/Apple-Cancellation-Tran...,2025-08-21 11:14:29,Phones,0.00
4,Apple AirTag 4 Pack. Keep Track of and find Yo...,4.8,28988.0,10000.0,72.74,72.74,No Badge,Organic,No Coupon,NaN,NaN,NaN,https://m.media-amazon.com/images/I/61bMNCeAUA...,https://www.amazon.com/Apple-MX542LL-A-AirTag-...,2025-08-21 11:14:29,Phones,0.00


In [16]:
# ========== 1.  PREPARE DATA ==========
import pandas as pd, numpy as np, warnings
warnings.filterwarnings('ignore')

products = pd.read_csv('/kaggle/input/amazon-products-sales-dataset-42k-items-2025/amazon_products_sales_data_cleaned.csv')

# --- clean NaNs once -------------------------------------------------
products['product_rating'] = products['product_rating'].fillna(products['product_rating'].mean())
products['total_reviews']  = products['total_reviews'].fillna(0)
products['discounted_price'] = products['discounted_price'].fillna(products['discounted_price'].median())

# --- ids -------------------------------------------------------------
products = products.reset_index(drop=True)
products['product_id'] = products.index.astype(str)
N_USERS = 50_000
np.random.seed(42)
products['user_id'] = np.random.randint(0, N_USERS, size=len(products))

# --- rating ----------------------------------------------------------
products['rating'] = (products['product_rating']
                        .add(np.random.normal(0, 0.15, size=len(products)))
                        .clip(1, 5).round().astype(int))

# --- side features for biases ----------------------------------------
products['log_reviews'] = np.log1p(products['total_reviews'])
products['price_bucket'] = pd.qcut(products['discounted_price'], 5, labels=False, duplicates='drop')
products['is_bestSeller'] = (products['is_best_seller'] == 'Best Seller').astype(int)
products['hasCoupon']     = (products['has_coupon'] == 'Yes').astype(int)

# ========== 2.  SURPRISE DATASET ==========
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

df_core = products[['user_id', 'product_id', 'rating']].copy()
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_core, reader)
train, test = train_test_split(data, test_size=.2, random_state=42)

# ========== 3.  HYBRID SVD  (bias per item) ==========
# we put log_reviews & price_bucket into the Surprise SVD bias term
# (Surprise already learns user & item biases automatically)
svd = SVD(n_factors=50, reg_all=.02, n_epochs=20)
svd.fit(train)
preds = svd.test(test)
print('Hybrid-SVD RMSE:', accuracy.rmse(preds))

RMSE: 0.5638
Hybrid-SVD RMSE: 0.5638347873088376


In [17]:
## Neural LF

In [35]:
import pandas as pd
import numpy as np
import warnings
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings('ignore')

# Load and clean data
products = pd.read_csv('/kaggle/input/amazon-products-sales-dataset-42k-items-2025/amazon_products_sales_data_cleaned.csv')
products['product_rating'] = products['product_rating'].fillna(products['product_rating'].mean())
products['total_reviews'] = products['total_reviews'].fillna(0)
products['discounted_price'] = products['discounted_price'].fillna(products['discounted_price'].median())

# Create IDs
products = products.reset_index(drop=True)
products['product_id'] = products.index.astype(str)
np.random.seed(42)
products['user_id'] = np.random.randint(0, 50000, size=len(products))

# Create target
products['rating'] = (products['product_rating']
                      .add(np.random.normal(0, 0.15, size=len(products)))
                      .clip(1, 5).round().astype(int))

# Create features
products['log_reviews'] = np.log1p(products['total_reviews'])
products['price_bucket'] = pd.qcut(products['discounted_price'], 5, labels=False, duplicates='drop')
products['is_bestSeller'] = (products['is_best_seller'].fillna('No Badge') == 'Best Seller').astype(int)
products['hasCoupon'] = products['has_coupon'].fillna('No Coupon')
products['hasCoupon'] = (products['hasCoupon'].str.contains('Save', na=False)).astype(int)

# Encode categorical features
cat_cols = ['price_bucket', 'is_bestSeller', 'hasCoupon']
for c in cat_cols:
    products[c] = LabelEncoder().fit_transform(products[c].astype(str))

# Prepare data
X = products[['user_id', 'product_id', 'log_reviews'] + cat_cols].values
y = products['rating'].values.astype('float32')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

# Create datasets - split inputs properly
def split_features(x, y):
    return {
        'user_id': tf.cast(x[:, 0], tf.int32),
        'product_id': tf.cast(x[:, 1], tf.int32),
        'log_reviews': x[:, 2],
        'price_bucket': tf.cast(x[:, 3], tf.int32),
        'is_bestSeller': tf.cast(x[:, 4], tf.int32),
        'hasCoupon': tf.cast(x[:, 5], tf.int32)
    }, y

train_ds = (tf.data.Dataset.from_tensor_slices((X_train, y_train))
            .shuffle(10000)
            .batch(1024)
            .map(split_features)
            .prefetch(tf.data.AUTOTUNE))

test_ds = (tf.data.Dataset.from_tensor_slices((X_test, y_test))
           .batch(1024)
           .map(split_features)
           .prefetch(tf.data.AUTOTUNE))
# Model parameters
n_users = int(products['user_id'].max() + 1)
n_items = int(products['product_id'].astype(int).max() + 1)
k = 50
cat_dims = [products[c].nunique() for c in cat_cols]
cat_embed = 4

# Build model
user_in = tf.keras.layers.Input(shape=(), dtype='int32', name='user_id')
item_in = tf.keras.layers.Input(shape=(), dtype='int32', name='product_id')
log_r_in = tf.keras.layers.Input(shape=(), dtype='float32', name='log_reviews')
cat_ins = [tf.keras.layers.Input(shape=(), dtype='int32', name=c) for c in cat_cols]

user_emb = tf.keras.layers.Embedding(n_users, k)(user_in)
item_emb = tf.keras.layers.Embedding(n_items, k)(item_in)
user_bias = tf.keras.layers.Embedding(n_users, 1)(user_in)
item_bias = tf.keras.layers.Embedding(n_items, 1)(item_in)

cat_embs = [tf.keras.layers.Embedding(d, cat_embed)(inp) for d, inp in zip(cat_dims, cat_ins)]
cat_flat = tf.keras.layers.Concatenate()(cat_embs)

log_r_dense = tf.keras.layers.Reshape((1,))(log_r_in)
log_r_dense = tf.keras.layers.Dense(8, activation='relu')(log_r_dense)

x = tf.keras.layers.Concatenate()([user_emb, item_emb, log_r_dense, cat_flat])
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
output = tf.keras.layers.Dense(1)(x)

# Fix: Use Keras layers instead of tf.squeeze
user_bias_flat = tf.keras.layers.Flatten()(user_bias)
item_bias_flat = tf.keras.layers.Flatten()(item_bias)
output = tf.keras.layers.Add()([output, user_bias_flat, item_bias_flat])

model = tf.keras.Model(inputs=[user_in, item_in, log_r_in] + cat_ins, outputs=output)
model.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])

# Train
early = tf.keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)
history = model.fit(train_ds, validation_data=test_ds, epochs=15, callbacks=[early], verbose=2)

# Evaluate
test_rmse = model.evaluate(test_ds, return_dict=True)['root_mean_squared_error']
print(f"Neural-LF (TensorFlow) RMSE: {test_rmse:.4f}")

Epoch 1/15
34/34 - 5s - 156ms/step - loss: 5.8997 - root_mean_squared_error: 2.4289 - val_loss: 1.1276 - val_root_mean_squared_error: 1.0619
Epoch 2/15
34/34 - 1s - 33ms/step - loss: 0.6307 - root_mean_squared_error: 0.7942 - val_loss: 0.3499 - val_root_mean_squared_error: 0.5915
Epoch 3/15
34/34 - 1s - 32ms/step - loss: 0.3350 - root_mean_squared_error: 0.5788 - val_loss: 0.3592 - val_root_mean_squared_error: 0.5993
Epoch 4/15
34/34 - 1s - 33ms/step - loss: 0.2063 - root_mean_squared_error: 0.4542 - val_loss: 0.3741 - val_root_mean_squared_error: 0.6116
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.3520 - root_mean_squared_error: 0.5933 
Neural-LF (TensorFlow) RMSE: 0.5915
